# Use Flux to generate Synthetic Data

In [ ]:
from groundedvision.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

### Install dependencies
pip install git+https://github.com/huggingface/diffusers.git

pip install --upgrade transformers accelerate bitsandbytes

## 4-bit quantized fully local

In [ ]:
import torch
from diffusers import Flux2Pipeline, AutoModel
from transformers import Mistral3ForConditionalGeneration
from diffusers.utils import load_image

repo_id = "diffusers/FLUX.2-dev-bnb-4bit" #quantized text-encoder and DiT. VAE still in bf16
device = "cuda:0"
torch_dtype = torch.bfloat16

text_encoder = Mistral3ForConditionalGeneration.from_pretrained(
    repo_id, subfolder="text_encoder", torch_dtype=torch.bfloat16, device_map="cpu"
)
dit = AutoModel.from_pretrained(
    repo_id, subfolder="transformer", torch_dtype=torch.bfloat16, device_map="cpu"
)
pipe = Flux2Pipeline.from_pretrained(
    repo_id, text_encoder=text_encoder, transformer=dit, torch_dtype=torch_dtype
)
pipe.enable_model_cpu_offload()

In [ ]:
prompt = "Realistic  photograph of the interior of a skyscrapper construction site. The interior should contain Tarp, half painted walls, open electric outlets. The wall of the site should have text `Track3D` on it and it has a color gradient that start with #FF5733 at the top and transitions to #33FF57 at the bottom."
#cat_image = load_image("https://huggingface.co/spaces/zerogpu-aoti/FLUX.1-Kontext-Dev-fp8-dynamic/resolve/main/cat.png")
image = pipe(
    prompt=prompt,
    #image=[cat_image] #multi-image input
    generator=torch.Generator(device=device).manual_seed(42),
    num_inference_steps=50,
    guidance_scale=4,
).images[0]

image.save("track3d_output.png")

## 4-bit quantized with remote Text encoder

In [ ]:
import torch
from diffusers import Flux2Pipeline
from diffusers.utils import load_image
from huggingface_hub import get_token
import requests
import io

repo_id = "diffusers/FLUX.2-dev-bnb-4bit" #quantized text-encoder and DiT. VAE still in bf16
device = "cuda:0"
torch_dtype = torch.bfloat16

def remote_text_encoder(prompts):
    response = requests.post(
        "https://remote-text-encoder-flux-2.huggingface.co/predict",
        json={"prompt": prompts},
        headers={
            "Authorization": f"Bearer {get_token()}",
            "Content-Type": "application/json"
        }
    )
    prompt_embeds = torch.load(io.BytesIO(response.content))

    return prompt_embeds.to(device)

pipe = Flux2Pipeline.from_pretrained(
    repo_id, text_encoder=None, torch_dtype=torch_dtype
).to(device)

In [ ]:

#cat_image = load_image("https://huggingface.co/spaces/zerogpu-aoti/FLUX.1-Kontext-Dev-fp8-dynamic/resolve/main/cat.png")
image = pipe(
    prompt_embeds=remote_text_encoder(prompt),
    #image=[cat_image] #optional multi-image input
    generator=torch.Generator(device=device).manual_seed(42),
    num_inference_steps=50, #28 steps can be a good trade-off
    guidance_scale=4,
).images[0]

image.save("track3d_output_2.png")

In [ ]:
prompts = [
    "Realistic  photograph of the interior of a skyscrapper construction site. The interior should contain Tarp, half painted walls, open electric outlets. The wall of the site should have text `Track3D` on it",
    "Realistic photograph of a skyscraper interior during sunset. Golden light beams cutting through dust motes in the air. Unfinished concrete floors reflect the light. A large concrete pillar in the foreground features the text 'Track3D' stenciled in black.",
    "Interior construction site at night, lit by harsh portable halogen work lights. High contrast shadows. Exposed steel beams and scattered debris. The text 'Track3D' is spray-painted in neon orange on a bare drywall partition.",
    "Soft, diffused lighting coming through floor-to-ceiling windows of an unfinished high-rise. Piles of drywall sheets and a yellow ladder. The words 'Track3D' appear as a decal on the glass window pane looking out at the city.",
    "Cinematic shot of a dimly lit construction corridor. Haze and sawdust in the air. Exposed HVAC ducts hanging from the ceiling. On a distressed brick wall, the text 'Track3D' is visible in white chalk style.",
    "A brightly lit, nearly finished medical facility construction site. White primed walls, protective plastic covering the floors. A digital rendering or poster taped to the wall displays the text 'Track3D'.",
    "Interior shot focusing on metal stud framing without drywall. The skeleton of the building is visible. Wires hanging loosely. A safety banner hanging between two studs reads 'Track3D'.",
    "Wide angle shot of a commercial space with open electrical boxes and plumbing pipes exposed. Plywood subflooring. The text 'Track3D' is projected onto the back wall using a light projector.",
    "Close-up perspective of a wall with fresh drywall mud and tape seams. A putty knife rests on a ledge. The text 'Track3D' is written into the wet plaster on the wall.",
    "Low angle shot looking up at an open ceiling grid with exposed sprinkler systems and insulation. A hanging construction sign in the foreground displays 'Track3D'.",
    "Eye-level shot of a worker's perspective (no people visible) looking at half-tiled floors. Boxes of ceramic tiles stacked. The text 'Track3D' is printed on the side of a large wooden crate.",
    "Symmetrical shot looking down a long, unfinished hallway. Cables snaking along the floor. Door frames installed but no doors. The text 'Track3D' is painted on the far end wall.",
    "High-angle fisheye lens view looking down from a mezzanine into a large construction atrium. Scaffolding towers rising up. The text 'Track3D' is on a large tarp draped over the scaffolding.",
    "Extreme close-up of an open electrical outlet with colorful wires protruding. The background is blurred (bokeh), but the wall behind clearly shows the text 'Track3D'.",
    "Framing the shot through a rough door opening into a larger room. Dust sheets covering equipment. The text 'Track3D' is on a whiteboard leaning against the wall in the center of the room.",
    "View from inside an unfinished elevator lobby looking toward the open shaft doors. Safety barriers in place. The concrete wall above the elevator opening has 'Track3D' stenciled in red.",
    "Interior of a loft renovation. Exposed old brick walls and new timber framing. Sawdust on the floor. The text 'Track3D' is painted in a vintage mural style on the brick.",
    "Modern office construction. Reflection of the interior seen in a glass partition. Polished concrete floors. The text 'Track3D' is etched into a frosted glass divider.",
    "Chaotic interior shot featuring pink insulation batting stuffed into walls and coils of Romex wire. The text 'Track3D' is printed on the vapor barrier plastic covering the wall.",
    "Brutalist style construction interior. Massive concrete columns and ceiling. Industrial look. 'Track3D' is laser-etched into the main concrete column.",
    "A room covered entirely in drop cloths. Paint buckets and rollers in the foreground. One wall is painted a vibrant accent color, with the text 'Track3D' masked out in white tape."
]

## Generate sample under construction photos

In [ ]:
from tqdm.notebook import tqdm


for idx, prompt_text in enumerate(tqdm(prompts)):
    image = pipe(
        prompt_embeds=remote_text_encoder(prompt_text),
        #image=[cat_image] #optional multi-image input
        generator=torch.Generator(device=device).manual_seed(42),
        num_inference_steps=50, #28 steps can be a good trade-off
        guidance_scale=4,
    ).images[0]
    
    image.save(f"track3d_output_{idx}.png")

### Generate Sample Side by Side Construction Photos with progress

In [ ]:
prompts = [
    "A realistic split-screen photograph showing the progress of a skyscraper office interior. LEFT PANEL (Earlier State): Exposed metal stud framing, hanging electrical wires, and a blue tarp covering a large window opening. The text 'Track 3D' is spray-painted orange on the concrete subfloor. RIGHT PANEL (Later State): The same view with drywall installed and primed white, electrical outlets with cover plates on, and a finished glass window unit installed, letting in clear daylight. The 'Track 3D' paint on the floor is partially covered by drywall dust.",

    "A diptych photograph illustrating construction timeline in a hallway. LEFT SIDE: Rough framing, temporary string lighting casting long shadows, and open plumbing pipes in the ceiling. A 'Track 3D' safety sign leans against a stud. RIGHT SIDE: The same hallway with fitted drywall ceiling grid, permanent recessed lights turned on, and walls painted a base coat of grey. The 'Track 3D' sign is now taped neatly to the painted wall.",

    "Side-by-side comparison shot of a high-rise corner office. LEFT VIEW: Bare concrete columns, piles of insulation batting on the floor, and open electrical junction boxes. 'Track 3D' is stenciled on the concrete column. RIGHT VIEW: The same corner with insulation installed behind drywall, closed junction boxes, and a primer coat of paint on the walls. The 'Track 3D' stencil on the column is still visible but slightly faded by dust.",

    "A split-view photo looking down a long corridor. LEFT: Messy plywood flooring, coils of Romex wire hanging from the ceiling, and rough door openings. 'Track 3D' written in chalk on a temporary plywood barrier. RIGHT: Polished concrete flooring, wires hidden above a partially installed drop ceiling, and finished metal door frames in place. The 'Track 3D' text is now on a printed paper sign taped to a finished door frame.",

    "Before-and-after comparison of an elevator lobby. LEFT PANEL: Open elevator shafts with safety netting, exposed cinder block walls, and temporary work lights. A large 'Track 3D' decal is on a wooden cable spool. RIGHT PANEL: Elevator doors installed with protective film, cinder blocks covered with scratch-coat plaster, and brighter, permanent overhead lighting. The wooden spool with the 'Track 3D' decal is still present but moved to the side.",

    "A photorealistic diptych of a future conference room. LEFT SIDE: A skeleton of ceiling channels, floor-to-ceiling glass partition frames without glass, and scattered tools. 'Track 3D' is written on a whiteboard leaning against the back wall. RIGHT SIDE: Acoustic ceiling tiles partially installed, glass panes fitted into the partition frames, and the walls painted a light blue accent color. The whiteboard with 'Track 3D' remains in the same spot.",

    "Split-screen view of a kitchen breakroom area under construction. LEFT: Roughed-in plumbing for sinks sticking out of the wall, open electrical boxes for appliances, and bare drywall seams. 'Track 3D' is marked with tape on the floor where an island will go. RIGHT: Cabinets partially installed, electrical outlets covered with white plates, and the first coat of paint on the walls. The 'Track 3D' floor tape is peeled up at the edges.",

    "Comparison photograph of a large open-plan office floor. LEFT PANEL: A forest of support jacks holding up concrete beams, dust in the air, and no interior walls. A banner hanging from the ceiling reads 'Track 3D Site Safety'. RIGHT PANEL: Support jacks removed, metal stud framing defining cubicle areas, and the air is clearer. The 'Track 3D Site Safety' banner is still hanging in the same location.",

    "A side-by-side look at a utility closet. LEFT VIEW: A chaotic jumble of exposed conduits, open breaker panels with colorful wires spilling out. 'Track 3D' written in marker on a junction box. RIGHT VIEW: Wires neatly organized into conduit, breaker panel covers installed, and the walls painted utility grey. The 'Track 3D' writing on the junction box is still visible.",

    "A split photograph focusing on a large window wall. LEFT: No glass, just open structural steel framing exposed to the elements, with tarps flapping in the wind. A 'Track 3D' sticker is on a steel beam. RIGHT: Floor-to-ceiling curtain wall glass installed, sealing the room from the outside. The interior walls framing the window are insulated and boarded. The 'Track 3D' sticker remains on the steel beam behind the new glass.",

    "Before-and-after diptych of a staircase core. LEFT SIDE: Raw concrete stairs with temporary wooden handrails, exposed standpipes, and dim lighting. 'Track 3D' stenciled in yellow on a riser. RIGHT SIDE: Steel handrails installed, standpipes painted red, and bright functional stairwell lighting turned on. The 'Track 3D' stencil is covered by a protective cardboard tread.",

    "Split-screen photo of a reception area. LEFT PANEL: A rough plywood mockup of a reception desk, bare concrete floors, and open ceiling. 'Track 3D' is spray-painted on the plywood desk. RIGHT PANEL: The reception desk clad in a stone veneer, stone floor tiles partially laid, and a finished drywall ceiling with recessed lighting features. The 'Track 3D' spray paint is covered by the new stone cladding.",

    "Comparison view of a bathroom facility. LEFT: Open floor trenches for plumbing, bare studs, and no fixtures. A blueprint taped to a stud has 'Track 3D' stamped on it. RIGHT: Trenches filled with concrete, moisture-resistant drywall installed, and rough-in valves for toilets visible. The blueprint with 'Track 3D' is still taped to the wall.",

    "A diptych showing ceiling progress. LEFT VIEW: Looking up at a chaotic mess of HVAC ducts, sprinkler pipes, and wiring bundles. A 'Track 3D' tag hangs from a wire. RIGHT VIEW: The same upward view showing a neatly suspended ceiling grid partially filled with acoustic tiles, hiding most of the utilities. The 'Track 3D' tag is peeking out from above a tile.",

    "Split-screen photo of a mechanical room. LEFT PANEL: Large equipment covered in plastic sheeting, uninsulated pipes, and debris on the floor. 'Track 3D' written on the plastic sheeting. RIGHT PANEL: Plastic removed from equipment, pipes insulated with white jackets, and the floor swept clean. The plastic sheeting with 'Track 3D' is bundled in a corner.",

    "A before-and-after shot of a retail space entrance. LEFT: Temporary plywood doors, rough framing for a storefront display, and open electrical for signage. 'Track 3D' painted on the plywood door. RIGHT: Aluminum-framed glass double doors installed, the storefront framing enclosed with drywall, and electrical wires capped. The 'Track 3D' painted plywood door is leaning against a side wall.",

    "A split-view photograph of a server room under construction. LEFT SIDE: Empty server racks covered in dust, raised flooring grid visible without tiles, and wire trays empty. A 'Track 3D' sticker on the rack. RIGHT SIDE: Server racks wiped clean, raised floor tiles installed, and bundles of blue ethernet cables running into the trays. The 'Track 3D' sticker is still on the rack.",

    "Comparison of an executive office suite. LEFT PANEL: Metal framing for a coffered ceiling, rough window openings, and a concrete floor. 'Track 3D' written on the floor. RIGHT PANEL: Drywall installed on the coffered ceiling structure, wood window trim installed, and a plywood subfloor laid over the concrete. The 'Track 3D' writing on the concrete is now covered by plywood.",

    "A diptych showing wall finishing. LEFT VIEW: A long wall showing bare drywall with visible screw holes and seams. A bucket of joint compound with 'Track 3D' written on it sits on the floor. RIGHT VIEW: The same wall with all seams taped and mudded, showing white stripes against the grey drywall, ready for sanding. The 'Track 3D' bucket is still there, now open with a trowel inside.",

    "Split-screen photo of a rooftop patio construction. LEFT: Exposed roofing membrane, stacks of pavers, and unfinished parapet walls. A 'Track 3D' banner tied to a temporary railing. RIGHT: Pavers installed on pedestals, parapet walls capped with metal coping, and glass railings installed. The 'Track 3D' banner is removed, but a small 'Track 3D' plaque is mounted near the door."
]

In [ ]:
from tqdm.notebook import tqdm


for idx, prompt_text in enumerate(tqdm(prompts)):
    image = pipe(
        prompt_embeds=remote_text_encoder(prompt_text),
        #image=[cat_image] #optional multi-image input
        generator=torch.Generator(device=device).manual_seed(42),
        num_inference_steps=50, #28 steps can be a good trade-off
        guidance_scale=4,
    ).images[0]
    
    image.save(f"track3d_progress_output_{idx}.png")

### Construction Categories Generation

In [ ]:
import pandas as pd

# Read Excel file
df = pd.read_excel(f'{PROCESSED_DATA_DIR}/Construction_Categories.xlsx')


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df_subset = df[['Class', 'Category', 'Stages']]

In [ ]:

markdown = df_subset.to_markdown(index=False)

In [ ]:
print(markdown)

In [ ]:

# Write to file
with open('categories.md', 'w', encoding='utf-8') as f:
    f.write(markdown)

print("✓ Conversion complete! Saved to categories.md")
print("\nPreview:")
print(markdown[:500])  # Show first 500 characters